<a href="https://colab.research.google.com/github/ryali93/utilitarios/blob/master/extractData/extractDataSenamhi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Descargar data de la pagina de SENAMHI

La ruta de la que se descarga es la siguiente:

https://www.senamhi.gob.pe/?&p=estaciones

Además, se está usando como referencia el archivo JSON

https://raw.githubusercontent.com/ryali93/UNMSM_programacion/master/data/estaciones.json


In [0]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [0]:
def extraeDataSenamhi(code_estation, year_month, type_est, estado):
    url = "https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/_dato_esta_tipo02.php"
    new_url = "{}?estaciones={}&CBOFiltro={}&t_e={}&estado={}".format(url,
                                                                     code_estation,
                                                                     year_month,
                                                                     type_est,
                                                                     estado)
    s = requests.get(new_url)
    html = s.text
    soup = BeautifulSoup(html, 'html.parser')
    tables = [
        [
            [td.get_text(strip=True) for td in tr.find_all('td')] 
            for tr in table.find_all('tr')
        ] 
        for table in soup.find_all('table')
    ]
    
    if len(tables[1][0]) != len(tables[1][1]):
      tables[1][0].insert(1, "TEMP_MIN")
      tables[1][0][2] = "TEMP_MAX"
      df = pd.DataFrame(tables[1][2:], columns=tables[1][0])
    else:
      df = pd.DataFrame(tables[1][1:], columns=tables[1][0])
    return df

In [0]:
def main(code_estation, year_month, type_est, estado):
    ldates = [str(y) + str(x).zfill(2) for x in range(1,13) for y in range(year_month[0], year_month[1])]
    ldates.sort()
    data = []
    for l in ldates:
        print(l)
        try:
            df = extraeDataSenamhi(code_estation, l, type_est, estado)
            if type(data) == list:
                data = pd.DataFrame({}, columns = list(df.columns))
            else:
                data = data.append(df)
            print(l)
        except:
            pass
    return data

Es necesario cambiar los datos del código que se encuentra debajo por lo siguiente:

***main(CODIGO, [AÑO_INI, AÑO_FIN], TIPO, ESTADO)***



In [0]:
# Esta es la función importante
df = main("104080", [2017, 2020], "M", "REAL")
df
# Si se requiere exportar a csv
# df.to_csv ('4724851A.csv', index = None, header=True)
# df

Si se desea ver los datos de las estaciones, usar el código a continuación

In [0]:
import json
import requests 

s = requests.get('https://raw.githubusercontent.com/ryali93/UNMSM_programacion/master/data/estaciones.json')
estaciones = s.text

data = json.loads(estaciones)
data["estaciones"]